In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Define the true function
def true_function(x):
    """y = sin(π*x/2) + x/2"""
    return np.sin((np.pi/2) * x) + x/2


def linear_interpolate_sid(x_data, y_data, x_query):
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    x_query = np.asarray(x_query)
    
    if not np.all(x_data[:-1] <= x_data[1:]):
        sort_indices = np.argsort(x_data)
        x_data = x_data[sort_indices]
        y_data = y_data[sort_indices]
    
    y_interp = np.zeros_like(x_query, dtype=float)
    
    for idx, x in enumerate(x_query):
        if x <= x_data[0]:
            y_interp[idx] = y_data[0]
            continue
        if x >= x_data[-1]:
            y_interp[idx] = y_data[-1]
            continue
        for i in range(len(x_data) - 1):
            if x_data[i] <= x <= x_data[i + 1]:
                x0 = x_data[i]
                x1 = x_data[i + 1]
                y0 = y_data[i]
                y1 = y_data[i + 1]
                y_interp[idx] = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
                break
    return y_interp


# ============================================================================
# READ DATA FROM FILE
# ============================================================================

file_path = r"D:\\Physics\\Computational Physics and Astrophysics\\Interpolation\\function_data.txt"

x_data = []
y_data = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        parts = line.split()
        if len(parts) >= 2:
            x_data.append(float(parts[0]))
            y_data.append(float(parts[1]))

x_data = np.array(x_data)
y_data = np.array(y_data)

# ============================================================================
# GENERATE 100 EVALUATION POINTS (10x HIGHER RESOLUTION)
# ============================================================================

x_eval = np.linspace(x_data.min(), x_data.max(), len(x_data) * 10 - 9)

print("="*80)
print("INTERPOLATION SETUP")
print("="*80)
print(f"Original data points: {len(x_data)}")
print(f"Interpolation points: {len(x_eval)}")

# Linear interpolation using SID method
y_linear = linear_interpolate_sid(x_data, y_data, x_eval)

# Cubic spline interpolation
cubic_spline = CubicSpline(x_data, y_data, bc_type='natural')
y_cubic = cubic_spline(x_eval)

# Get true values from actual function at interpolation points
y_true = true_function(x_eval)
